# Finetuning LLM using Unsloth

Let's try a simple finetuning phi-3.5-mini-instruct on thinking dataset using unsloth

In [1]:
!nvidia-smi

Fri Sep 20 13:23:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:35:00.0 Off |                    0 |
| N/A   71C    P0              33W /  72W |  11308MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Setup

In [ ]:
%%capture
# get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
!pip install accelerate -U

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3.5-mini-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.0.dev0.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 128,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Data preparation

In [5]:
from datasets import load_dataset

ds = load_dataset("SkunkworksAI/reasoning-0.01",split = "train")

In [6]:
ds[0]

{'instruction': 'If a die is rolled three times, what is the probability of getting a sum of 11? None',
 'reasoning': '1. Understand the problem: We need to find the probability of getting a sum of 11 when rolling a die three times.\n2. Calculate total possible outcomes: A die has 6 faces, so for each roll, there are 6 possibilities. For three rolls, the total possible outcomes are 6^3 = 216.\n3. Identify favorable outcomes: List all combinations of rolls that result in a sum of 11. There are 18 such combinations.\n4. Calculate probability: Divide the number of favorable outcomes by the total possible outcomes: 18 / 216 = 1/12.\n5. Conclusion: The probability of getting a sum of 11 when rolling a die three times is 1/12.',
 'output': "To solve this problem, we need to find the number of favorable outcomes (getting a sum of 11) and divide it by the total possible outcomes when rolling a die three times.\n\nFirst, let's find the total possible outcomes. Since a die has six faces, there a

In [7]:
from unsloth.chat_templates import get_chat_template

def formatting_prompts(example):
    reasoning = ""
    t = [{
        "role":"user",
        "content":f"{example['instruction']}"},
        {
        "role":"assistant",
        "content":f"<thinking>{example['reasoning_chains'][0:-1]}</thinking> {example['reasoning_chains'][-1]['thought']}"
    }]
    return t

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3",
    mapping = {"role" : "user", "content" : "content", "user" : "user", "assistant" : "assistant"}
)

def formatting_prompts_func(example):
    conversations = formatting_prompts(example)
    texts = tokenizer.apply_chat_template(conversations, tokenize = False, add_generation_prompt = False)
    return { "text" : texts, }

In [8]:
dataset = ds.map(formatting_prompts_func, batched = False,)

In [9]:
dataset['text'][0]

"<|user|>\nIf a die is rolled three times, what is the probability of getting a sum of 11? None<|end|>\n<|assistant|>\n<thinking>[{'step': 1, 'thought': 'Understand the problem: We need to find the probability of getting a sum of 11 when rolling a die three times.'}, {'step': 2, 'thought': 'Calculate total possible outcomes: A die has 6 faces, so for each roll, there are 6 possibilities. For three rolls, the total possible outcomes are 6^3 = 216.'}, {'step': 3, 'thought': 'Identify favorable outcomes: List all combinations of rolls that result in a sum of 11. There are 18 such combinations.'}, {'step': 4, 'thought': 'Calculate probability: Divide the number of favorable outcomes by the total possible outcomes: 18 / 216 = 1/12.'}]</thinking> Conclusion: The probability of getting a sum of 11 when rolling a die three times is 1/12.<|end|>\n"

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 4,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        save_strategy = "steps",
        save_steps=30,
        per_device_train_batch_size = 32,
        gradient_accumulation_steps = 2,
        warmup_steps = 10,
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs_lora_r128_lalpha128",
    ),
)

max_steps is given, it will override any value given in num_train_epochs


In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 29,857 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 32 | Gradient Accumulation steps = 2
\        /    Total batch size = 64 | Total steps = 100
 "-____-"     Number of trainable parameters = 239,075,328
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anto-grimaldi7 (anto-grimaldi7-italy). Use `wandb login --relogin` to force relogin


Step,Training Loss
1,0.980100
2,1.037400
3,0.992800
4,0.991300
5,0.831400
6,0.818700
7,0.820300
8,0.749100
9,0.705400
10,0.712500


## Inference

In [12]:
from unsloth.chat_templates import get_chat_template


def formatting_prompts(example):
    reasoning = ""
    t = [{
        "role":"user",
        "content":f"{example['instruction']}"},
        {
        "role":"assistant",
        "content":f"<thinking>{example['reasoning_chains'][0:-1]}</thinking> {example['reasoning_chains'][-1]['thought']}"
    }]
    return t

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3",
    mapping = {"role" : "user", "content" : "content", "user" : "user", "assistant" : "assistant"}
)

def formatting_prompts_func(example):
    conversations = formatting_prompts(example)
    texts = tokenizer.apply_chat_template(conversations, tokenize = False, add_generation_prompt = False)
    return { "text" : texts, }


In [13]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32064, 3072)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=128, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=128, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, 

In [14]:
messages = [
    {"role": "user", "content": " If five cats can catch five mice in five minutes, how long will it take one cat to catch one mouse?"},
]

In [15]:
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

In [16]:
outputs = model.generate(input_ids = inputs, max_new_tokens = 4089, use_cache = True)
response = tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [17]:
import re
import json 
import ast

answer = response[0].split("</thinking>")[1]
result = re.search(r'<thinking>(.*?)</thinking>', response[0])
if result:
    t = result.group(1)
    
    outs = t.split(",")
    for d in outs:
        print(d)
print(answer)

[{'step': 1
 'thought': 'The problem states that five cats can catch five mice in five minutes.'}
 {'step': 2
 'thought': 'This implies that each cat is catching one mouse in five minutes.'}
 {'step': 3
 'thought': 'Therefore
 the rate at which one cat catches a mouse is one mouse per five minutes.'}
 {'step': 4
 'thought': 'The question then asks how long it will take one cat to catch one mouse.'}
 {'step': 5
 'thought': 'Since we've established that one cat catches one mouse in five minutes
 the answer to the question is five minutes.'}]
 Therefore, it will take one cat five minutes to catch one mouse.<|end|><|endoftext|>


## Text Streamer 

In [18]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference


messages = [
    {"role": "user", "content": "There is a barrel with no lid and some wine in it. “This barrel of wine is more than half full,” says the woman. “No, it's not,” says the man. “It’s less than half full.” Without any measuring implements and without removing any wine from the barrel, how can they easily determine who is correct?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024, use_cache = True)

<thinking>[{'step': 1, 'thought': 'The problem is about determining who is correct between the woman and the man about the amount of wine in the barrel.'}, {'step': 2, 'thought': 'The barrel has no lid, so we cannot see the amount of wine directly.'}, {'step': 3, 'thought': 'The woman says the wine is more than half full, and the man says it is less than half full.'}, {'step': 4, 'thought': 'The problem states that we cannot remove any wine from the barrel, so we cannot use a measuring instrument to determine the amount of wine.'}, {'step': 5, 'thought': 'However, we can use the fact that the barrel has no lid to our advantage.'}, {'step': 6, 'thought': 'If the wine is more than half full, the barrel will be heavier than when it was empty.'}, {'step': 7, 'thought': 'If the wine is less than half full, the barrel will be lighter than when it was empty.'}, {'step': 8, 'thought': 'Therefore, we can determine who is correct by comparing the weight of the barrel to its weight when it was em

#### As you can see from the outputs, the model has adopted the 'thinking' structure in multiple steps when providing the final answer.